In [1]:
# import required libraries and read the data
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import math

df=pd.read_csv('Credit.csv')
df.head()

,Income,Limit,Rating,Cards,Age,Education,OwnHome,Student,Married,Region,Balance
0,14.891,3606,283,2,34,11,No,No,Yes,South,333
1,106.025,6645,483,3,82,15,Yes,Yes,Yes,West,903
2,104.593,7075,514,4,71,11,No,No,No,West,580
3,148.924,9504,681,3,36,11,Yes,No,No,West,964
4,55.882,4897,357,2,68,16,No,No,Yes,South,331


In [2]:
df.shape

(400, 11)

In [3]:
# check for missing values
df.isnull().sum()

Income       0
Limit        0
Rating       0
Cards        0
Age          0
Education    0
OwnHome      0
Student      0
Married      0
Region       0
Balance      0
dtype: int64

In [5]:
# exploratory data analysis
df.describe()

,Income,Limit,Rating,Cards,Age,Education,Balance
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,45.218885,4735.600000,354.940000,2.957500,55.667500,13.450000,520.015000
std,35.244273,2308.198848,154.724143,1.371275,17.249807,3.125207,459.758877
min,10.354000,855.000000,93.000000,1.000000,23.000000,5.000000,0.000000
25%,21.007250,3088.000000,247.250000,2.000000,41.750000,11.000000,68.750000
50%,33.115500,4622.500000,344.000000,3.000000,56.000000,14.000000,459.500000
75%,57.470750,5872.750000,437.250000,4.000000,70.000000,16.000000,863.000000
max,186.634000,13913.000000,982.000000,9.000000,98.000000,20.000000,1999.000000


In [6]:
# get binary variables of categorical columns
df=pd.get_dummies(df)
df.head()

,Income,Limit,Rating,Cards,Age,Education,Balance,OwnHome_No,OwnHome_Yes,Student_No,Student_Yes,Married_No,Married_Yes,Region_East,Region_South,Region_West
0,14.891,3606,283,2,34,11,333,1,0,1,0,0,1,0,1,0
1,106.025,6645,483,3,82,15,903,0,1,0,1,0,1,0,0,1
2,104.593,7075,514,4,71,11,580,1,0,1,0,1,0,0,0,1
3,148.924,9504,681,3,36,11,964,0,1,1,0,1,0,0,0,1
4,55.882,4897,357,2,68,16,331,1,0,1,0,0,1,0,1,0


In [7]:
# create correlation matrix
corr = df.corr()
corr.style.background_gradient()

,Income,Limit,Rating,Cards,Age,Education,Balance,OwnHome_No,OwnHome_Yes,Student_No,Student_Yes,Married_No,Married_Yes,Region_East,Region_South,Region_West
Income,1.000000,0.792088,0.791378,-0.018273,0.175338,-0.027692,0.463656,0.010738,-0.010738,-0.019632,0.019632,-0.035652,0.035652,0.040132,-0.019701,-0.017137
Limit,0.792088,1.000000,0.996880,0.010231,0.100888,-0.023549,0.861697,-0.009397,0.009397,0.006015,-0.006015,-0.031155,0.031155,0.036320,-0.003081,-0.032427
Rating,0.791378,0.996880,1.000000,0.053239,0.103165,-0.030136,0.863625,-0.008885,0.008885,0.002028,-0.002028,-0.036751,0.036751,0.037598,-0.001070,-0.035999
Cards,-0.018273,0.010231,0.053239,1.000000,0.042948,-0.051084,0.086456,0.022658,-0.022658,0.026164,-0.026164,0.009695,-0.009695,0.000878,-0.005631,0.005591
Age,0.175338,0.100888,0.103165,0.042948,1.000000,0.003619,0.001835,-0.004015,0.004015,0.029844,-0.029844,0.073136,-0.073136,0.061169,-0.000822,-0.059623
Education,-0.027692,-0.023549,-0.030136,-0.051084,0.003619,1.000000,-0.008062,0.005049,-0.005049,-0.072085,0.072085,-0.048911,0.048911,0.013827,-0.037725,0.029586
Balance,0.463656,0.861697,0.863625,0.086456,0.001835,-0.008062,1.000000,-0.021474,0.021474,-0.259018,0.259018,0.005673,-0.005673,0.013720,-0.003288,-0.009812
OwnHome_No,0.010738,-0.009397,-0.008885,0.022658,-0.004015,0.005049,-0.021474,1.000000,-1.000000,0.055034,-0.055034,0.012452,-0.012452,0.014288,0.009831,-0.025425
OwnHome_Yes,-0.010738,0.009397,0.008885,-0.022658,0.004015,-0.005049,0.021474,-1.000000,1.000000,-0.055034,0.055034,-0.012452,0.012452,-0.014288,-0.009831,0.025425
Student_No,-0.019632,0.006015,0.002028,0.026164,0.029844,-0.072085,-0.259018,0.055034,-0.055034,1.000000,-1.000000,-0.076974,0.076974,-0.001931,0.048334,-0.053534


In [8]:
# set dependent and independent variables
X = df[['Rating', 'Student_Yes']].copy()
Y = df['Balance'].copy()

In [9]:
# split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [10]:
# run OLS Regression model on Train set
X_train = sm.add_constant(X_train) 
est = sm.OLS(Y_train,X_train).fit()
predictions = est.predict() 
print(est.summary())
print("\nAverage error: {:.2f}.".format(math.sqrt(est.mse_resid)))

                            OLS Regression Results                            
Dep. Variable:                Balance   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.813
Method:                 Least Squares   F-statistic:                     606.1
Date:                Fri, 05 Jan 2024   Prob (F-statistic):          6.80e-102
Time:                        05:20:38   Log-Likelihood:                -1881.6
No. Observations:                 280   AIC:                             3769.
Df Residuals:                     277   BIC:                             3780.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -434.0290     29.730    -14.599      

In [11]:
# use K-Fold cross validation 
kf =KFold(n_splits=5, shuffle=True, random_state=0)
lr = LinearRegression()
CVscores= cross_val_score(lr, X_train, Y_train, scoring='r2', cv=kf)
print('Cross validation R-Squared ', CVscores)
print('Cross validation R-Squared mean:', CVscores.mean())

Cross validation R-Squared  [0.81296453 0.79167939 0.79268713 0.78159709 0.85738448]
Cross validation R-Squared mean: 0.8072625241253961


K-Fold cross validation is used to evaluate machine learning models by providing for more accurate measure of performance. It splits the dataset into equally sized subsets (folds) where one fold acts as test set and the others as train sets. This process repeats until all folds acted as a test fold. Scores are retained after each iteration and then averaged to asses the overall performance of the model. 

The average R^2 from the cross validation is 0.807 which is 0.07 less than the score from the model on Train set. This indicates that the model has an overfitting problem, but because the difference is very small it could be ignored

In [13]:
# check for multicollinearity, VIF for the features must be less than 2
variables = est.model.exog
vif = pd.DataFrame() 
vif["VIF Factor"] = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif["features"] = X_train.columns
print('VIF: {}'.format(vif))

VIF:    VIF Factor     features
0    6.085428        const
1    1.005715       Rating
2    1.005715  Student_Yes


In [14]:
# run the regression on the Test set
X_test = sm.add_constant(X_test)
est = sm.OLS(Y_test,X_test).fit()
predictions = est.predict()
print(est.summary())
print("\nAverage error: {:.2f}.".format(math.sqrt(est.mse_resid)))

                            OLS Regression Results                            
Dep. Variable:                Balance   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     256.7
Date:                Fri, 05 Jan 2024   Prob (F-statistic):           1.62e-43
Time:                        05:24:57   Log-Likelihood:                -800.76
No. Observations:                 120   AIC:                             1608.
Df Residuals:                     117   BIC:                             1616.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -434.4279     48.857     -8.892      

In [15]:
# multicollinearity check
variables = est.model.exog
vif = pd.DataFrame() 
vif["VIF Factor"] = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif["features"] = X_test.columns
print('VIF: {}'.format(vif))

VIF:    VIF Factor     features
0    7.628032        const
1    1.034316       Rating
2    1.034316  Student_Yes


**Interpret the regression results:**

**R^2** - 0.814 meaning that 81.4% of change in price can be explained by Rating and Student_Yes

**Adj. R^2** - Adj. R^2 is lower than R^2 indicating some variables are not contributing to the model's R^2 properly, but they are very close so it should not be a problem

**Prob (F-statistic)** - Prob (F-statistic) is 1.62x10^-43 which is less than 0.05 showing the model is linear and significant

**coefficients** - for 1 unit increase in Rating the Balance will increase by 2.61, and relative to the Student_No, Student_Yes have on average higher Balance by 354.49 

**p-values** - p-values for both independent variables are less than 0.05 meaning they are both statistically significant

**Average Error** - The Average Error is 193.78 meaning that on average the model is 193.78 off

The average errors are different between the Test and Train sets, and the R^2 for the both models is the same but the Adjusted R^2 for the Train set is 0.02 higher than for the Test set indicating very slight overfitting problem

The model is generalizable to the unseen data because even though there is a difference between the Test set and Train set in Average Error and Adj. R^2 resulting in an overfitting problem, but the difference is very small and the R^2 is same for the two sets, therefore, the model should generalize well to any new data.